In [1]:
%%time
! python A0_explore_data.py

CPU times: user 44 ms, sys: 16 ms, total: 60 ms
Wall time: 2.84 s


In [2]:
%%time
! cp /input/data_ori/* /input/ 
#! python A1_sample_dataset.py

CPU times: user 576 ms, sys: 184 ms, total: 760 ms
Wall time: 33.9 s


In [3]:
%%time
! python B1_create_item_table.py

Iteration is stopped
Iteration is stopped
Iteration is stopped
CPU times: user 4.58 s, sys: 1.25 s, total: 5.83 s
Wall time: 4min 20s


In [4]:
%%time
! python B2_create_user_table.py

Iteration is stopped
Iteration is stopped
Iteration is stopped
CPU times: user 12 s, sys: 3.74 s, total: 15.8 s
Wall time: 12min 31s


# 京东JData算法大赛(1)-数据清洗

  比赛的题目是高潜用户的购买意向的预测,从机器学习的角度来讲我们可以认为这是一个二分类的任务.那么我们就是尝试去构建自己的正负样本.  
  由于我们拿到的是原始数据,里面存在很多噪声,因而第一步我们先要对数据清洗,比如说:  
- 去掉只有购买记录的用户(没有可用的历史浏览等记录来预测用户将来的购买意向)  
- 去掉浏览量很大而购买量很少的用户(惰性用户或爬虫用户)
- 去掉最后5(7)天没有记录(交互)的商品和用户
- ......

为了能够进行上述清洗,在此首先构造了简单的用户(user)行为特征和商品(item)行为行为特征,对应于两张表user_table和item_table
- **user_table**特征包括:  
  user_id(用户id),age(年龄),sex(性别),  
  user_lv_cd(用户级别),browse_num(浏览数),  
  addcart_num(加购数),delcart_num(删购数),  
  buy_num(购买数),favor_num(收藏数),  
  click_num(点击数),buy_addcart_ratio(购买加购转化率),  
  buy_browse_ratio(购买浏览转化率),  
  buy_click_ratio(购买点击转化率),  
  buy_favor_ratio(购买收藏转化率)  
- **item_table**特征包括:  
  sku_id(商品id),attr1,attr2,  
  attr3,cate,brand,browse_num,  
  addcart_num,delcart_num,  
  buy_num,favor_num,click_num,  
  buy_addcart_ratio,buy_browse_ratio,  
  buy_click_ratio,buy_favor_ratio,  
  comment_num(评论数),  
  has_bad_comment(是否有差评),  
  bad_comment_rate(差评率)

In [5]:
# 定义文件名
ACTION_201602_FILE = "/input/JData_Action_201602.csv"
ACTION_201603_FILE = "/input/JData_Action_201603.csv"
ACTION_201604_FILE = "/input/JData_Action_201604.csv"
COMMENT_FILE = "/input/JData_Comment.csv"
PRODUCT_FILE = "/input/JData_Product.csv"
USER_FILE = "/input/JData_User.csv"
NEW_USER_FILE = "/input/JData_User_New.csv"
USER_TABLE_FILE = "/input/user_table.csv"
ITEM_TABLE_FILE = "/input/item_table.csv"

In [6]:
# 导入相关包
import pandas as pd
import numpy as np
from collections import Counter

import os

接下来以构建user_table为例,item_table与之类似.

In [7]:
# 功能函数: 对每一个user分组的数据进行统计
def add_type_count(group):
    behavior_type = group.type.astype(int)
    # 用户行为类别
    type_cnt = Counter(behavior_type)
    # 1: 浏览 2: 加购 3: 删除
    # 4: 购买 5: 收藏 6: 点击
    group['browse_num'] = type_cnt[1]
    group['addcart_num'] = type_cnt[2]
    group['delcart_num'] = type_cnt[3]
    group['buy_num'] = type_cnt[4]
    group['favor_num'] = type_cnt[5]
    group['click_num'] = type_cnt[6]

    return group[['user_id', 'browse_num', 'addcart_num',
                  'delcart_num', 'buy_num', 'favor_num',
                  'click_num']]

由于用户行为数据量较大,一次性读入可能造成内存错误(Memory Error),因而使用pandas的分块(chunk)读取.

In [8]:
#　对action数据进行统计
# 根据自己的需求调节chunk_size大小
def get_from_action_data(fname, chunk_size=100000):
    reader = pd.read_csv(fname, header=0, iterator=True)
    chunks = []
    loop = True
    while loop:
        try:
            # 只读取user_id和type两个字段
            chunk = reader.get_chunk(chunk_size)[["user_id", "type"]]
            chunks.append(chunk)
        except StopIteration:
            loop = False
            print("Iteration is stopped")
    # 将块拼接为pandas dataframe格式
    df_ac = pd.concat(chunks, ignore_index=True)
    # 按user_id分组，对每一组进行统计
    df_ac = df_ac.groupby(['user_id'], as_index=False).apply(add_type_count)
    # 将重复的行丢弃
    df_ac = df_ac.drop_duplicates('user_id')

    return df_ac

In [9]:
# 将各个action数据的统计量进行聚合
def merge_action_data():
    df_ac = []
    df_ac.append(get_from_action_data(fname=ACTION_201602_FILE))
    df_ac.append(get_from_action_data(fname=ACTION_201603_FILE))
    df_ac.append(get_from_action_data(fname=ACTION_201604_FILE))

    df_ac = pd.concat(df_ac, ignore_index=True)
    # 用户在不同action表中统计量求和
    df_ac = df_ac.groupby(['user_id'], as_index=False).sum()
    #　构造转化率字段
    df_ac['buy_addcart_ratio'] = df_ac['buy_num'] / df_ac['addcart_num']
    df_ac['buy_browse_ratio'] = df_ac['buy_num'] / df_ac['browse_num']
    df_ac['buy_click_ratio'] = df_ac['buy_num'] / df_ac['click_num']
    df_ac['buy_favor_ratio'] = df_ac['buy_num'] / df_ac['favor_num']
    
    # 将大于１的转化率字段置为１(100%)
    df_ac.ix[df_ac['buy_addcart_ratio'] > 1., 'buy_addcart_ratio'] = 1.
    df_ac.ix[df_ac['buy_browse_ratio'] > 1., 'buy_browse_ratio'] = 1.
    df_ac.ix[df_ac['buy_click_ratio'] > 1., 'buy_click_ratio'] = 1.
    df_ac.ix[df_ac['buy_favor_ratio'] > 1., 'buy_favor_ratio'] = 1.

    return df_ac

In [10]:
#　从JData_User表中抽取需要的字段
def get_from_jdata_user():
    df_usr = pd.read_csv(NEW_USER_FILE, header=0)
    df_usr = df_usr[["user_id", "age", "sex", "user_lv_cd"]]
    return df_usr

构建user table的main函数

In [11]:
user_base = get_from_jdata_user()
user_behavior = merge_action_data()

# 连接成一张表，类似于SQL的左连接(left join)
user_behavior = pd.merge(user_base, user_behavior, on=['user_id'], how='left')
# 保存为user_table.csv
user_behavior.to_csv(USER_TABLE_FILE, index=False)

Iteration is stopped
Iteration is stopped
Iteration is stopped


item table的构建过程与user table的类似，这里直接运行我事先写好的脚本．

### 数据清洗

In [12]:
df_usr = pd.read_csv(USER_TABLE_FILE, header=0)

# 输出前5行数据
df_usr.head(5)

,user_id,age,sex,user_lv_cd,browse_num,addcart_num,delcart_num,buy_num,favor_num,click_num,buy_addcart_ratio,buy_browse_ratio,buy_click_ratio,buy_favor_ratio
0,200001,5,2.0,5,212.0,22.0,13.0,1.0,0.0,414.0,0.045455,0.004717,0.002415,1.0
1,200002,-1,0.0,1,238.0,1.0,0.0,0.0,0.0,484.0,0.000000,0.000000,0.000000,NaN
2,200003,3,1.0,4,221.0,4.0,1.0,0.0,1.0,420.0,0.000000,0.000000,0.000000,0.0
3,200004,-1,2.0,1,52.0,0.0,0.0,0.0,0.0,61.0,NaN,0.000000,0.000000,NaN
4,200005,1,0.0,4,106.0,2.0,3.0,1.0,2.0,161.0,0.500000,0.009434,0.006211,0.5


In [13]:
# 输出购买数为0的后5条数据
df_usr[df_usr['buy_num'] == 0].tail(5)

,user_id,age,sex,user_lv_cd,browse_num,addcart_num,delcart_num,buy_num,favor_num,click_num,buy_addcart_ratio,buy_browse_ratio,buy_click_ratio,buy_favor_ratio
105314,305315,3,2.0,2,12.0,0.0,0.0,0.0,0.0,19.0,NaN,0.0,0.0,NaN
105315,305316,3,1.0,2,20.0,0.0,0.0,0.0,0.0,16.0,NaN,0.0,0.0,NaN
105318,305319,2,0.0,3,590.0,3.0,0.0,0.0,0.0,1189.0,0.0,0.0,0.0,NaN
105319,305320,3,2.0,2,68.0,1.0,0.0,0.0,0.0,150.0,0.0,0.0,0.0,NaN
105320,305321,3,2.0,3,38.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN,NaN


In [14]:
# 删除购买数为0的用户
df_usr = df_usr[df_usr['buy_num'] != 0]

In [15]:
# 输出格式设置，保留三位小数
pd.options.display.float_format = '{:,.3f}'.format

# 输出user table的统计信息,包括
# 总数，均值，方差，最小值，1/4分位数，1/2分位数，3/4分位数，最大值
df_usr.describe()

,user_id,age,sex,user_lv_cd,browse_num,addcart_num,delcart_num,buy_num,favor_num,click_num,buy_addcart_ratio,buy_browse_ratio,buy_click_ratio,buy_favor_ratio
count,"29,626.000","29,626.000","29,624.000","29,626.000","29,485.000","29,485.000","29,485.000","29,485.000","29,485.000","29,485.000","29,485.000","29,485.000","29,485.000","29,485.000"
mean,"250,738.984",2.011,1.025,4.268,302.475,10.525,4.673,1.636,1.677,486.637,0.360,0.018,0.030,0.862
std,"29,987.018",1.240,0.959,0.812,391.525,14.301,7.568,1.412,4.584,658.654,0.320,0.038,0.136,0.287
min,"200,001.000",-1.000,0.000,1.000,1.000,0.000,0.000,1.000,0.000,0.000,0.004,0.000,0.000,0.010
25%,"225,040.500",2.000,0.000,4.000,76.000,3.000,0.000,1.000,0.000,116.000,0.118,0.004,0.002,1.000
50%,"249,144.500",2.000,1.000,4.000,178.000,6.000,2.000,1.000,0.000,282.000,0.250,0.008,0.005,1.000
75%,"276,253.250",3.000,2.000,5.000,381.000,13.000,6.000,2.000,1.000,604.000,0.500,0.018,0.012,1.000
max,"305,318.000",5.000,2.000,5.000,"7,605.000",288.000,178.000,50.000,96.000,"15,302.000",1.000,1.000,1.000,1.000


In [16]:
# 查看购买数为１而浏览数大于8000的用户
df_usr[(df_usr['buy_num'] < 2) & (df_usr['browse_num'] > 6000)]

,user_id,age,sex,user_lv_cd,browse_num,addcart_num,delcart_num,buy_num,favor_num,click_num,buy_addcart_ratio,buy_browse_ratio,buy_click_ratio,buy_favor_ratio
7080,207081,2,0.000,5,"6,230.000",48.000,40.000,1.000,96.000,"9,733.000",0.021,0.000,0.000,0.010
12089,212090,3,0.000,5,"6,229.000",185.000,178.000,1.000,1.000,"9,821.000",0.005,0.000,0.000,1.000
